# Gradient Optimization 梯度优化

## 问题描述

Consider an aircraft with the rectangular wing described in the previous assignment. 
The drag coefficient for the aircraft is given as,
$$
C_{D}=\frac{0.03062702}{S}+k C_{f} \frac{S_{\mathrm{wet}}}{S}+\frac{C_{L}^{2}}{\pi A e}
$$
Where first term corresponds to the parasite drag of the fuselage and the other two terms are for the wing, as given in the previous assignment.

The weight of the wing can be approximated as,
$$
W_{w}=45.42 S+8.71 \times 10^{-5} \frac{N_{\mathrm{ult}} b^{3} \sqrt{W_{0} W}}{S(t / c)}
$$
where $ N_{ult} $ is the ultimate load factor considered in the structural sizing, and $ t / c $ is the average thickness to chord ratio. If the weight of the aircraft excluding the wing is $ W_{0} $, the total weight of the aircraft is,
$$
W = W_{0} + W_{w}
$$
Note that the weight of the wing (2) depends on itself through $ W $, so you will need to iterate to find the correct weights.

Once you have the aircraft weight, you can find the required lift coefficient for level flight,
$$
C_{L}=\frac{2 W}{\rho V^{2} S}
$$
Then you can compute the lift to drag ratio $ L / D=C_{L} / C_{D} $ and the total drag for level flight is
$$
D=\frac{W}{L / D}
$$
The optimization problem is to minimize $ D $ with respect to the aspect ratio $ A $ and the wing area $ S $. The values for all the constants new to this assignment are listed in Table 1 ; refer to the previous assignment for the remaining constants.
\begin{array}{lrll}
\text { Quantity } & \text { Value } & \text { Units } & \text { Description } \\
\hline W_{0} & 4940 & \mathrm{~N} & \text { weight of the aircraft excluding the wing } \\
N_{\text {ult }} & 2.5 & - & \text { ultimate load factor } \\
t / c & 0.12 & - & \text { average thickness to chord ratio }
\end{array}

In [91]:
import numpy as np
import copy

def Cal_Re(c, rho=1.23, V=35.0, mu=17.8e-6):
    """Calculate Reynolds number."""
    return rho*V*c/mu

def Cal_Cf(Re):
    """Calculate the skin friction coefficient"""
    return 0.074/(Re**0.2)

def Cal_Cd(A, k=1.2, S=11.8, S_ratio=2.05, Cl=0.3, e=0.96):
    """Calculate the total drag coefficient as the function of A"""
    b = np.sqrt(A*S)
    c = S/b
    Re = Cal_Re(c)
    Cf = Cal_Cf(Re)
    return 0.03062702/S+k*Cf*S_ratio+Cl**2/(np.pi*A*e)

def Cal_Ww(S, W, b, Nult = 2.5, t_c = 0.12, W0 = 4940):
    """Calculate the weight of wing"""
    return 45.42*S + 8.71e-5*Nult*(b**3)*np.sqrt(W0*W)/(S*t_c)

def Cal_D(A, S, rho=1.23, V=35.0, Nult = 2.5, t_c = 0.12, W0 = 4940):
    """Calculate the drag"""
    b = np.sqrt(A*S)
    error = 10000
    W = 0
    while(error>1e-8):
        W_old = W
        W = W0 + Cal_Ww(S, W, b, Nult = Nult, t_c = t_c, W0 = W0)
        error = W - W_old
    Cl = 2 * W / (rho*V**2*S)
    Cd = Cal_Cd(A=A, S=S, Cl=Cl)
    return W/(Cl/Cd)

### 梯度函数


$$
g(A, S) = \frac{\partial D}{\partial A} + \frac{\partial D}{\partial S}
$$
其中，
$$
\frac{\partial D}{\partial A} = \frac{\partial}{\partial A} \left( \frac{W}{L / D} \right) 
= \frac{\partial}{\partial A} \left( \frac{W C_{D}}{C_{L}} \right)
= \frac{W}{C_{L}} \frac{\partial C_{D}}{\partial A}
= \frac{\rho V^{2} S}{2} \left( k \frac{S_{wet}}{S} \frac{\partial C_{f}}{\partial A} - \frac{C_{L}^{2}}{\pi A^{2} e} \right)
= \frac{\rho V^{2} S}{2} \left( k \frac{S_{wet}}{S} \frac{0.2 \times 0.074}{{Re}^{1.2}} \frac{\rho V \sqrt{S}}{2 \mu A \sqrt{A}} - \frac{C_{L}^{2}}{\pi A^{2} e} \right)
$$

$$
\frac{\partial D}{\partial S} = \frac{\partial}{\partial S} \left( \frac{W}{L / D} \right) 
= \frac{\partial}{\partial S} \left( \frac{W C_{D}}{C_{L}} \right)
= \frac{\partial \frac{1}{2} \rho V^{2} S C_{D}}{\partial S}
= \frac{\rho V^{2} S}{2} \left( \frac{\partial C_{D}}{\partial S} S + C_{D} \right)
$$
其中
$$
\frac{\partial C_{D}}{\partial S}
= - \frac{0.03062702}{S^{2}} + k \frac{S_{wet}}{S} \frac{\partial C_{f}}{\partial S} + \frac{2 C_{L}}{\pi A e} \frac{\partial C_{L}}{\partial S}
$$
$$
\frac{\partial C_{f}}{\partial S} 
= - \frac{0.2 \times 0.074}{{Re}^{1.2}} \frac{\partial Re}{\partial S}
= - \frac{0.2 \times 0.074}{{Re}^{1.2}} \frac{1}{2 \mu \sqrt{A} \sqrt{S}} 
$$

$$
\frac{\partial C_{L}}{\partial S}
= \frac{2}{\rho S^{2}} \frac{\partial}{\partial S} \left( \frac{W}{S} \right)
= \frac{2}{\rho S^{2}} \left[ -\frac{W}{S^{2}} + \frac{1}{S} \frac{\partial W}{\partial S} \right]
$$

由$W = W_{0} + W_{w}$
$$
\frac{\partial W}{\partial S} 
= \frac{\partial W_{w}}{\partial S} 
= 45.42 + 8.71 \times 10^{-5} \frac{N_{ult} b^{3}}{t/c} \left[ -\frac{\sqrt{W_{0} W}}{S^{2}} + \frac{1}{2} \sqrt{\frac{W_{0}}{W}} \frac{\partial W}{\partial S} \right]
$$
因此有：
$$
\frac{\partial W}{\partial S} 
= \frac{45.42 + 8.71 \times 10^{-5} \frac{N_{ult} b^{3}}{t/c} \left[ -\frac{\sqrt{W_{0} W}}{S^{2}} \right]}{8.71 \times 10^{-5} \frac{N_{ult} b^{3}}{t/c} \left[  \frac{1}{2} \sqrt{\frac{W_{0}}{W}} \frac{\partial W}{\partial S} \right]}
$$

## Steepest Descent Method

#### 一维线搜索

In [92]:
class WolfeConditionsSearch():
    """A line search that satisfies the strong Wolfe conditions."""
    def __init__(self, function, mu1=1e-4, mu2=0.9):
        self.function = function
        self.mu1 = mu1
        self.mu2 = mu2
    
    def gfunction(self, x, h=0.0001):
        return (self.function(x+h)-self.function(x-h))/(2*h)

    def lineSearch(self, a_step = 1 , iter_max=2000):      
        # Search the space of a
        a_best = 0
        a0 = 0
        a = a0 + a_step
        find = False
        iteration = 1    
        while(iteration<iter_max):
            if(self.function(a) > self.function(a0) + self.mu1*a*self.gfunction(a0)):
                a_min = a - a_step
                a_max = a 
                break
            if(abs(self.gfunction(a)) < abs(self.mu2*self.gfunction(a0)) and self.gfunction(a)<0):
                a_best = a
                find = True 
                break
            if(self.gfunction(a)>0):
                a_min = a - a_step
                a_max = a 
                break
            a += a_step
            iteration += 1
        if(iteration>=iter_max):
            print("There is no minimum point")
            return 404

        if(find):
            return a_best
            
        # reduce the space of a
        iteration = 1
        while(iteration<iter_max):  
            a = (a_min + a_max) / 2.0
            if(self.function(a) > self.function(a0)+self.mu1*a*self.gfunction(a0)):
                a_max = a 
                continue
            if(abs(self.gfunction(a)) < abs(self.mu2*self.gfunction(a0)) and self.gfunction(a)<0):
                a_best = a
                find = True 
                break
            if(self.gfunction(a)*(a_max - a_min)>0):
                a_max = a 
            elif(self.gfunction(a)*(a_max - a_min)<0):
                a_min = a
            iteration += 1
        
        if(find):
            return a_best
        else:
            print("The iteration has reach the maximum number")
            return 404


#### 最陡牛顿下降法

In [93]:
class GradientOptimization():
    '''The Gradient Optimization'''
    def __init__(self, function, x_init = np.zeros([2]), dim = 2, low = np.zeros([2]), up = np.ones([2])):
        self.function = function
        self.dim = dim
        self.low = low
        self.up = up
        self.x = x_init
        self.xbest = x_init
        self.ybest = self.function(x_init)
    
    def gfunction(self, x, h = 0.00001):
        g = np.zeros(len(x))
        for i in range(len(x)):
            xh2 = copy.deepcopy(x)
            xh2[i] = x[i] + h
            xh1 = copy.deepcopy(x)
            xh1[i] = x[i] - h
            g[i] = ( self.function(xh2) - self.function(xh1) ) /(2*h)
        return g

    def SteepestDescentMethod(self,  eg = 1e-6, ea = 1e-6, er = 0.01, iter_max = 2000):
        """ The Steep Descent Method. """
        iteration = 1
        itercount = 0
        while(iteration < iter_max ):
            g = self.gfunction(x=self.x)
            if(np.sqrt(sum(g**2)<=eg)):
                return self.xbest, self.ybest
            
            p = - g /np.sqrt(sum(g**2))
            afunction = lambda a: self.function(self.x + a * p)

            finda = WolfeConditionsSearch(afunction)
            a_best = finda.lineSearch() 
            if(abs(self.function(self.x)-self.function(self.x + a_best*p)) <= ea+er*abs(self.function(self.x))):
                itercount += 1
            else:
                itercount = 0
            
            if(itercount == 2):
                self.xbest = self.x
                self.ybest = self.function(self.x)
                return self.xbest, self.ybest
            else:
                self.x = self.x + a_best*p
                iteration += 1

def function(x):
    return Cal_D(A=x[0], S=x[1])

x_init = np.array([14.24945028, 11.76839883])
opt = GradientOptimization(function=function, x_init=x_init)
print(opt.SteepestDescentMethod())


(array([14.24166232, 11.70638595]), 258.2709428896324)
